In [ ]:
import hashlib
from io import BytesIO

import util
from test_framework.key import ECKey, ECPubKey, generate_schnorr_nonce
from test_framework.musig import generate_musig_key, aggregate_schnorr_nonces, sign_musig, aggregate_musig_signatures
from test_framework.script import TapLeaf, TapTree, CTransaction, TaprootSignatureHash
from test_framework.address import program_to_witness
from test_framework.messages import CTransaction, COutPoint, CTxIn, CTxOut, CScriptWitness, CTxInWitness

# 2.4 Tapscript


Tapscript is a updated Bitcoin script language which is evaluated when the a Taproot output is spent along the _script path_. 

Tapscript retains most of the opcodes and evaluation rules applicable to v0 witness scripts, but includes several notable updates described in part 1 of this chapter.

* __Part 1: Script Updates__
    * Signature opcode updates.
        * Schnorr verification.
        * Checksigadd.
    * Future Versioning
        * Tapscript
        * Opcodes

In part 2, we propose the following TapScript descriptors, which provide the most common script locking conditions.

* __Part 2: Tapscript Descriptors__ (Proposed)
    * `Pay-to-pubkey` descriptors:
        * pk(key)
        * pkhash(key, hash)
        * pkolder(key, delay)
        * pkhasholder(key, hash, delay)
    * `Checksigadd` descriptors:
        * csa(n, key, ...)
        * csahash(n, key, ..., hash)
        * csaolder(n, key, ..., delay)
        * csahasholder(n, key, ..., hash, delay)


## Part 1: Script Updates.

### Schnorr verification with signature opcodes.

The signature opcodes consume the same stack arguments as in Segwit v0, but now verify signatures defined in [bip-schnorr](https://github.com/sipa/bips/blob/bip-schnorr/bip-schnorr.mediawiki).

* OP_CHECKSIG
    * Stack arguments consumed: `[public key] [bip-schnorr signature]`
    * Pushes 0x01 on success or 0x00 on failure onto the stack.
    * Requires an empty signature 0x00 to fail.
    
    
* OP_CHECKSIGVERIFY
    * Stack arguments consumed: `[public key] [bip-schnorr signature]`
    * Continues with next opcode or fails script evaluation.




#### 2.4.1 Example: Pay-to-pubkey tapscript. 

* The pay-to-pubkey tapscript consist of the following script operations:
    * `[pk] [checksig]`
* `TapLeaf.construct_pk(ECPubKey)` contructs a pk tapscript.
* `TapLeaf.sat` describes witness elements required to satisfy the tapscript.

In [ ]:
# Generate Keypair.
privkey0 = ECKey()
privkey0.generate()
pubkey0 = privkey0.get_pubkey()

# Generate Tapscript.
pk_tapscript = TapLeaf()
pk_tapscript.construct_pk(pubkey0)

print("Tapscript operations:")
for op in pk_tapscript.script:
    print(op.hex()) if isinstance(op, bytes) else print(op)

print("\nSatisfying witness elements:")
for element, value in pk_tapscript.sat:
    print("Witness element type is {}.".format(element))
    print("Signature corresponds to pubkey {}.".format(value.hex()))

**Disabled checkmultisig opcodes:** Legacy n-of-m multisignature opcodes would previously check each of the n signatures against up to m public keys. This is inefficient and disables batch verification of schnorr signatures, since pubkey and signature pairs cannot be known prior to script execution.

* OP_CHECKMULTISIG
* OP_CHECKMULTISIGVERIFY

### Multisignatures with Checksigadd.

Tapscript replaces the previous checkmultisig signature operation with OP_CHECKSIGADD. 

This multisignature signature opcode requires the witness to provide a valid or invalid signature for each public key, thereby avoiding the need to waste signature verfication operations for each public key in n-of-m multisignature scripts.

* OP_CHECKSIGADD
    * Equivalent to: `[OP_ROT][OP_SWAP][OP_CHECKSIG][OP_ADD]`
    * Counted as one op towards 201 non-push opcode limit.
    * Enables multisig scripts as shown below.

![test](images/tapscript0.jpg)


### N-of-M Checksigadd tapscripts

Unlike legacy multisig, N-of-M checksigadd multisignature tapscripts will consume a stack element for each public key in the output script. This means that unused public keys must be evaluated against a zero witness element.

For example:
* Tapscript: `[pk0] [CHECKSIG] [PK1] [CHECKSIGADD] [PK2] [CHECKSIGADD] [2] [NUMEQUAL]`
* Witness: 
    * `[sig2]` `[sig1]` `[]`
    * `[sig2]` `[]` `[sig0]`
    * `[]` `[sig1]` `[sig0]`

The disadvantages of N-of-M Checksigadd multisignature scripts include
* Cost: unused public keys are paid by the spender.
* Privacy: unused public keys are revealed when tapscript is spent.

#### 2.4.2 Example: Generating a 2-of-3 checksigadd output. 

* We construct a 2-of-3 multisig output with `OP_CHECKSIGADD`. 

In [ ]:
# Generate Keypairs.
privkey0 = ECKey()
privkey1 = ECKey()
privkey2 = ECKey()
privkey0.generate()
privkey1.generate()
privkey2.generate()
pubkey0 = privkey0.get_pubkey()
pubkey1 = privkey1.get_pubkey()
pubkey2 = privkey2.get_pubkey()

# Generate Tapscript.
csa_tapscript = TapLeaf()
csa_tapscript.construct_csa(2, [pubkey0, pubkey1, pubkey2])

print("CSA tapscript operations:")
for op in csa_tapscript.script:
    print(op.hex()) if isinstance(op, bytes) else print(op)

# Satisfying witness element.
print("\nSatisfying witness elements:")
for element, value in csa_tapscript.sat:
    print("Witness element type is {}.".format(element))
    print("Signature corresponds to pubkey {}.".format(value.hex()))
    

### N-of-M Multisig as  N-of-N checksigadd scripts

Alternatively, a N-of-M Multisig locking condition can be expressed with multiple N-of-N checksigadd tapscripts committed to the same taproot. This minimizes leakage of unused public keys and can be more cost-efficient for the spender.

Use the following convenience method to generate N-of-N checksigadd tapscripts from M public keys.
* `TapLeaf.generate_threshold_csa(n, [key_0, key_1, ..., key_m])`

![test](images/tapscript1.jpg)


#### 2.4.3 Example: 2-of-5 multisig expressed as 2-of-2 checkigadd tapscripts.

In [ ]:
# Generate Keypairs.
privkey0 = ECKey()
privkey1 = ECKey()
privkey2 = ECKey()
privkey3 = ECKey()
privkey4 = ECKey()
privkey0.generate()
privkey1.generate()
privkey2.generate()
privkey3.generate()
privkey4.generate()
pubkey0 = privkey0.get_pubkey()
pubkey1 = privkey1.get_pubkey()
pubkey2 = privkey2.get_pubkey()
pubkey3 = privkey3.get_pubkey()
pubkey4 = privkey4.get_pubkey()

# Generate Tapscripts.
pkv = [pubkey0, pubkey1, pubkey2, pubkey3, pubkey4]
tapscripts = TapLeaf.generate_threshold_csa(2, pkv)
    
for ts in tapscripts:
    print(ts.desc)

### Tapscript Versioning

Tapscript allows for different version types.

* Leaf version (commited to TapTree leaf node)
    * Initial version: `0xC0`
* Success opcodes (allow for future functionality).
    * 80, 98, 126-129, 131-134, 137-138, 141-142, 149-153, 187-254
    * Any of these opcodes end script evaluation successfully.

## Part 2: Tapscript Descriptors.

A tapscript descriptor is proposed to be encapsulated by a tapscript tag:
* `ts(pk(key))`, `ts(csa(key))`, ...
* The `ts` tag can be updated with new tap leaf versions.


### Pay-to-pubkey tapscript descriptors.

A pay-to-pubkey tapscript descriptor is proposed to have the following forms.  

* `ts(pk(key))`
    * Witness: `[signature]`
    
    
* `ts(pkhash(key, 20B-hash-digest))`
    * Witness: `[signature]`,`[32B-preimage]`
    
    
* `ts(pkolder(key, delay))`
    * Witness: `[signature]`
    * Spendable after delay.
    
    
* `ts(pkhasholder(key, 20B-hash-digest, delay))`
    * Witness: `[signature]`,`[32B-preimage]`
    * Spendable after delay.
    
We also provide pay-to-pubkey tapscript constructors for for the `TapLeaf` class. 

* `TapLeaf.construct_pk(ECPubKey)`
* `TapLeaf.construct_pkhash(ECPubKey, 20B-hash-digest)`
* `TapLeaf.construct_pkolder(ECPubKey, delay)`
* `TapLeaf.construct_pkhasholder(ECPubKey, 20B-hash-digest, delay)`


**Note:** Pubkeys in pay-to-pubkey tapscripts can generated with multi-party schemes such as MuSig.


#### 2.4.4 Example: Generating a `pkhasholder` tapscript.

We construct a pkhasholder tapscript with the following locking conditions.

* 2-of-2 MuSig public key.
* Hashlock with the the preimage `b'secret'`.
* Delay of 20 blocks.

In [ ]:
# Generate MuSig Key.
privkey0 = ECKey()
privkey1 = ECKey()
privkey0.generate()
privkey1.generate()
pubkey0 = privkey0.get_pubkey()
pubkey1 = privkey1.get_pubkey()
c_map, pk_musig = generate_musig_key([pubkey0, pubkey1])

# Generate pkolder tapscript.
pkhash_tapscript = TapLeaf()
pkhash_tapscript.construct_pkolder(pk_musig, 20)
print("Tapscript descriptor:", pkhash_tapscript.desc, "\n")

print("Tapscript operations:")
for op in pkhash_tapscript.script:
    print(op.hex()) if isinstance(op, bytes) else print(op)

print("\nSatisfying witness elements:")
for element, value in pkhash_tapscript.sat:
    print("{}, {}".format(element, value.hex()))


### Checksigadd tapscript descriptors.

A checksigadd tapscript descriptor is proposed to have the following forms.

* `ts(csa(n, key0, key1, ...))`
    * Witness: `[signature], [signature], ...`
    * Note: for n < m, empty signature elements (zero) must be provided.
    

* `ts(csaolder(n, key0, key1, ..., hash))`
    * Witness: `[signature], [signature], ..., [delay]`


* `ts(csahash(n, key0, key1, ..., hash, time))`
    * Witness: `[signature], [signature], ..., [32B pre-image]`


* `ts(csahasholder(n, key0, key1, ..., hash, time))`
    * Witness: `[signature], [signature], ..., [32B pre-image], [delay]`

We also provide checksigadd tapscript constructors for for the `TapLeaf` class. 

* `TapLeaf.construct_csa(n,ECPubKey, ECPubKey, ...)`
* `TapLeaf.construct_csahash(n, ECPubKey, ECPubKey, ..., 20B-hash-digest)`
* `TapLeaf.construct_csaolder(n, ECPubKey, ECPubKey, ..., delay)`
* `TapLeaf.construct_csahash(n, ECPubKey, ECPubKey, ..., 20B-hash-digest, delay)`


**Note:** Any single public key in CSA tapscripts can generated with multi-party schemes such as MuSig.


#### 2.4.5 _Programming Exercise:_ Generate a 2-of-2 `csahasholder` tapscript.

Construct a `csahasholder` tapscript with the following locking conditions.

* 2-of-2 public keys.
    * One single signer key.
    * One 2-of-2 MuSig key.
* Hashlock with the the preimage `sha256(b'secret')`.
* Delay of 20 blocks.

In [ ]:
# Generate key pairs.
privkey0 = # TODO: Implement
privkey1 = # TODO: Implement
privkey2 = # TODO: Implement
pk_musig = # TODO: Implement
print("MuSig pubkey: {}\n".format(pk_musig.get_bytes().hex()))

secret = # TODO: Implement
preimage = # TODO: Implement
digest = # TODO: Implement

# Construct Tapscript
csahasholder_tapscript = # TODO: Implement
# ...
print("Descriptor:", csahasholder_tapscript.desc, "\n")

print("Tapscript operations:")
for op in csahasholder_tapscript.script:
    print(op.hex()) if isinstance(op, bytes) else print(op)

print("\nSatisfying witness elements:")
for element, value in csahasholder_tapscript.sat:
    print("{}, {}".format(element, value.hex()))


#### 2.4.6 _Programming Exercise:_ Generate and spend a  tapscript.

**Part A:** First, spend 0.5 Bitcoin to this output from the Bitcoin Core wallet to a taproot output containing these tapscripts.

* `ts(pkolder(key0, 20))`
* `ts(csahash(key1, key2, ripemd160(sha(b'secret')))`
    * Where `key2` is a 2-of-2 MuSig key.

In [ ]:
# Generate Keys.
privkey_int = ECKey() 
privkey_int.generate()
pubkey_int = privkey_int.get_pubkey()
privkey0 = ECKey()
privkey1 = ECKey()
privkey2 = ECKey()
privkey3 = ECKey()
privkey0.generate()
privkey1.generate()
privkey2.generate()
privkey3.generate()
pubkey0 = privkey0.get_pubkey()
pubkey1 = privkey1.get_pubkey()
pubkey2 = privkey2.get_pubkey()
pubkey3 = privkey3.get_pubkey()

# Generate MuSig keys.
c_map, pk_musig = # TODO: Implement
privkey2_c = # TODO: Implement
privkey3_c = # TODO: Implement
print("MuSig pubkey: {}\n".format(pk_musig.get_bytes().hex()))

# Generate ts(pkolder(key0, 20)).
pkolder_ts = # TODO: Implement


# ts(csahash(2, key1, key23, b'secret').
csahash_ts = # TODO: Implement


# Satisfying witness elements.
print("Satisfying witness elements:")
for element, value in csahash_ts.sat:
    print("{}, {}".format(element, value.hex()))

# Construct TapTree.
taptree = # TODO: Implement


# Generate taproot output and segwit address.
taproot_script, tweak, control_map = taptree.construct()
pk_taproot_bytes = bytes(taproot_script[2:])
segwit_address = program_to_witness(1, pk_taproot_bytes)
print("\nBech32 address: {}".format(segwit_address))

***Start TestNode***

In [ ]:
test = util.TestWrapper()
test.setup(num_nodes=1)

***Generate Wallet Balance***

In [ ]:
test.nodes[0].generate(101)
balance = test.nodes[0].getbalance()
print(balance)

***Send funds from the wallet to the taproot output (Segwit Address).***

In [ ]:
# Send funds to taproot output.
txid = test.nodes[0].sendtoaddress(segwit_address, 0.5)
print("Funding tx:", txid)

# Deserialize wallet transaction.
tx = CTransaction()
tx_hex = test.nodes[0].getrawtransaction(txid)
tx.deserialize(BytesIO(bytes.fromhex(tx_hex)))
tx.rehash()

# Determine Output Index of Segwit V1 Output.
# (Wallet shuffles change output index.)
outputs = iter(tx.vout)
taproot_output = next(outputs)
index = 0

while (taproot_output.scriptPubKey != taproot_script):
    taproot_output = next(outputs)
    index += 1
output_value = taproot_output.nValue

**Part B:** Now construct a transaction that spends the taproot output along the csa script path.

In [ ]:
tx_script_spend = CTransaction()
tx_script_spend.nVersion = 2
tx_script_spend.nLockTime = 0
outpoint = COutPoint(tx.sha256, index)
tx_script_spend_input = CTxIn(outpoint = outpoint)
tx_script_spend.vin = [tx_script_spend_input]

# Generate new Bitcoin Core wallet address to send funds back to.
dest_addr = test.nodes[0].getnewaddress(address_type="bech32")
scriptpubkey = bytes.fromhex(test.nodes[0].getaddressinfo(dest_addr)['scriptPubKey'])

# Determine minimum fee required for mempool acceptance.
min_fee = int(test.nodes[0].getmempoolinfo()['mempoolminfee'] * 100000000)

# Complete output which returns funds to Bitcoin Core wallet.
dest_output= CTxOut(nValue=output_value-min_fee, scriptPubKey=scriptpubkey)
tx_script_spend.vout = [dest_output]

# Construct witness according to required satisfaction elements.
htv = [0, 1, 2, 3, 0x81, 0x82, 0x83]
sighash = TaprootSignatureHash(tx_script_spend, [taproot_output], htv[0], 0, scriptpath=True, tapscript=csahash_ts.script)

# Signature from privkey1.
sig1 = # TODO: Implement

# MuSig signature.
k2 = # TODO: Implement
k3 = # TODO: Implement
R2 = # TODO: Implement
R3 = # TODO: Implement
R_agg, negated = # TODO: Implement
sig_agg = # TODO: Implement

# Satisfying witness elements:
witness_elements = []
witness_elements.append() # TODO: Implement
witness_elements.append() # TODO: Implement
# ...

tx_script_spend.wit.vtxinwit.append(CTxInWitness())
tx_script_spend.wit.vtxinwit[0].scriptWitness.stack = witness_elements + [csahash_ts.script, control_map[csahash_ts.script]]
taproot_spend_str = tx_script_spend.serialize().hex()

# Serialize Schnorr transaction for broadcast.
tx_script_spend_str = tx_script_spend.serialize().hex()

# Test mempool acceptance.
print(test.nodes[0].testmempoolaccept([tx_script_spend_str]))


***Shutdown TestNodes***

In [ ]:
test.shutdown()